---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [159]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [160]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [161]:
def answer_one():
    a=(spam_data['target'].value_counts()[1])/(spam_data['target'].count())
    return float(a*100)

In [162]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [163]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():    
    v= CountVectorizer()
    v.fit(X_train)
    l=[(len(i),i) for i in v.get_feature_names()]# append length and word into the list
    
    l=sorted(l,reverse=True)#sort in descending order
    
    return l[0][1]

In [164]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [165]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    # fit_transform() method on the training set
    # transform() method on the test dataset
    v= CountVectorizer()
    X_train1 = v.fit_transform(X_train)
    X_test1= v.transform(X_test)
    
    m= MultinomialNB(alpha=0.1)
    m.fit(X_train1,y_train)
    pred = m.predict(X_test1)
    
    return roc_auc_score(y_test,pred)

In [166]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    import operator
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    smallest_tf_idfs_series = pd.Series(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[:20]],
                                        index = [X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[:20]],
                                                 feature_names[sorted_tfidf_index[:20]]]).sort_index().reset_index(level=0,drop=True)
    largest_tf_idfs_series = pd.Series(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[-20:]],
                                        index = [X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[-20:]],
                                                 feature_names[sorted_tfidf_index[-20:]]]).sort_index().reset_index(level=0,drop=True)
    return (smallest_tf_idfs_series, largest_tf_idfs_series)

In [207]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, blank        0.932702
 tick         0.980166
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [169]:
def answer_five():
    v= TfidfVectorizer(min_df=3)
    X_train1 = v.fit_transform(X_train)
    X_test1= v.transform(X_test)
    
    m= MultinomialNB(alpha=0.1)
    m.fit(X_train1,y_train)
    pred = m.predict(X_test1)
    
    return roc_auc_score(y_test,pred)

In [170]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [171]:
def answer_six():
    a=[ len(k) for k in spam_data[spam_data['target']==0]['text'] ]
    b=[ len(j) for j in spam_data[spam_data['target']==1]['text'] ]
    
    a1=sum(a)/len(a)
    b1=sum(b)/len(b)
    return (a1,b1)

In [172]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [173]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [174]:
from sklearn.svm import SVC

def answer_seven():
    v= TfidfVectorizer(min_df=5)
    X_train1 = v.fit_transform(X_train)
    X_test1= v.transform(X_test)
    
    X_train11 = add_feature(X_train1, X_train.str.len())
    X_test11 = add_feature(X_test1, X_test.str.len())
    
    m= SVC(C=10000)
    m.fit(X_train11,y_train)
    pred = m.predict(X_test11)
    
    return roc_auc_score(y_test,pred)

In [175]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [176]:
def answer_eight():
    a=0
    for k in spam_data[spam_data['target']==0]['text']:
     for i in k:
            if i.isdigit():
                a=a+1
    a1=a/(len(spam_data[spam_data['target']==0]['text']))
    
    b=0
    for k in spam_data[spam_data['target']==1]['text'] :
     for i in k:
            if i.isdigit():
                b=b+1
    b1=b/(len(spam_data[spam_data['target']==1]['text']))

    
    return (a1,b1)

In [177]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [204]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    v=CountVectorizer(min_df=5,ngram_range=[1,3])

    X_train1 = v.fit_transform(X_train)
    X_train2= add_feature(X_train1,[X_train.str.len(),X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])

    X_test1= v.transform(X_test)
    X_test2= add_feature(X_test1,[X_test.str.len(),X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])

    m= LogisticRegression(C=100)
    m.fit(X_train2, y_train)
    pred= m.predict(X_test2)

    return roc_auc_score(y_test,pred)


In [205]:
answer_nine()

0.96873673666876048

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [180]:
def answer_ten():
    import re
    a=[]
    for k in spam_data[spam_data['target']==0]['text']:
     a.append(re.findall("\W",k))
    b=[]
    for k in spam_data[spam_data['target']==1]['text'] :
     b.append(re.findall("\W",k))
    
    return (np.mean(list(map(len,a))),np.mean(list(map(len,b))))

In [181]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [202]:
def answer_eleven():
    v=CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb')
 
    X_train1 = v.fit_transform(X_train)
    condition0=X_train.str.len()
    condition1=X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    condition2=X_train.str.findall(r'(\W)').str.len()
    X_train2= add_feature(X_train1,[condition0,condition1,condition2])

    X_test1= v.transform(X_test)
    condition3=X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    condition4=X_test.str.findall(r'(\W)').str.len()
    X_test2= add_feature(X_test1,[X_test.str.len(),condition3,condition4])

    m= LogisticRegression(C=100)
    m.fit(X_train2, y_train)
    pred= m.predict(X_test2)
    score1=roc_auc_score(y_test, pred)
    
    feature_names = np.array(v.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = m.coef_[0].argsort()
    small= feature_names[sorted_coef_index[:10]]
    large= feature_names[sorted_coef_index[:-11:-1]]

    return (score1,small,large)


In [203]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U19'),
 array(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww',
        'ar'], 
       dtype='<U19'))